In [ ]:
#autor Rafael RR
import pandas as pd
import numpy as np
from sklearn import tree
import pickle
import re
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor




archivo_train="base.csv"
nombre_ia="rafa_bot.pkl"

In [115]:
#se crean las funciones de preproceso
def procesa_texto():
    cols=["tema","real"]
    cols_train="texto"
    col_layer="real"
    df=pd.read_csv(archivo_train,encoding ='latin1')    
    X_df = df
    X_df =sklearn.utils.shuffle(X_df)
    
    text = X_df[[cols_train]]
    Y = X_df[col_layer]
    
    l_texto=[]
    
    for i in range(0, len(text)):
        # Remueve caracteres especiales
        texto_aux = re.sub(r'\W', ' ', str(text["texto"][i]))
        
        # Sustituimos varios en espacios blanco por uno solo
        texto_aux = re.sub(r'\s+', ' ', texto_aux, flags=re.I)
        #print(text["texto"][i])
        
        # Convierte a minusculas
        texto_aux = texto_aux.lower()
        
        l_texto.append(texto_aux)
    
    # bolsa de palabras
    # remover stopwords
    vectorizer = CountVectorizer(stop_words=stopwords.words('spanish'))
    
    X = vectorizer.fit_transform(l_texto).toarray()
 
    tfidfconverter = TfidfTransformer()
    
    X = tfidfconverter.fit_transform(X).toarray()
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)
    modelo_tema=crea_modelo(data=X_train, etiqueta=y_train)
    
    
    estadistica_modelo(modelo_tema, X_test, y_test, nombre_ia)
    
    return [vectorizer, tfidfconverter]

def crea_modelo(data, etiqueta):
    #se encuentran varios modelos para que se realizen las pruebas del mejor modelo 
    #solo es necesario descomentar el modelo que se desea
    #modelo =SGDClassifier(loss='hinge', penalty='l2', 
     #                     alpha=1e-3, random_state=1, max_iter=2, tol=None)
    #modelo =MLPClassifier(solver='sgd', 
                  #alpha=1e-5,
                  #hidden_layer_sizes=(10, 20), random_state=1)
    modelo = BernoulliNB()
    #modelo = MLPClassifier(hidden_layer_sizes=(200,100,500), 
     #                      activation='relu', solver='adam', max_iter=5000)
    
    #modelo = RandomForestClassifier(n_estimators=1000, random_state=0)
    modelo.fit(data, etiqueta)
    return modelo

def estadistica_modelo(modelo, X_test, y_test, nombre_ia):
    y_pred = modelo.predict(X_test)
    
    #Evaluating the Model
    print ("---------------------------------------\n")
    print ("matriz de confunsion")
    print(confusion_matrix(y_test,y_pred))
    
    print ("---------------------------------------\n")
    print(classification_report(y_test,y_pred))
    
    print ("---------------------------------------\n")
    exito=int(accuracy_score(y_test, y_pred)*100)
    print("el exito del modelo es: "+str(exito)+ "%")
    
    print ("---------------------------------------\n")
    #guardamos el modelo 
    with open(nombre_ia, 'wb') as picklefile:
        pickle.dump(modelo,picklefile)
        
def usa_modelo(vectorizer, tfidfconverter, texto):
    #docs_new = ['hola soy nuevo', 'jonh titor']
    docs_new = [texto]
    X_new_counts = vectorizer.transform(docs_new)
    X_new_tfidf = tfidfconverter.transform(X_new_counts).toarray()
    
    with open(nombre_ia, 'rb') as training_model:
        modelo = pickle.load(training_model)
    
    predicted = modelo.predict(X_new_tfidf)
    return predicted

In [ ]:
# se entrena el modelo , espera unos minutos ..

l_contexto=procesa_texto()


In [117]:
print("introduce un texto que pueda ser una noticia falsa:")
texto=  input()
vectorizer=l_contexto[0]
tfidfconverter=l_contexto[1]
res=usa_modelo(vectorizer, tfidfconverter, texto)

if 0 == res[0]:
    print ("\n El bot determino que la noticia es falsa ")
else:
    print ("\n El bot determino que la noticia es verdadera ")


introduce un texto que pueda ser una noticia falsa:
sad asdas dasd

 El bot determino que la noticia es falsa 
